In [1]:
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import time
import json
from selenium.webdriver.chrome.service import Service
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

In [2]:
path=Service("C:\\Program Files (x86)\\Google\\Chrome\\Application\\chromedriver\\chromedriver.exe")
browser = webdriver.Chrome(chrome_options=options, service=path)


C:\Users\Christophe\AppData\Local\Temp\ipykernel_23844\816233213.py:2: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(chrome_options=options, service=path)


### GET informations about each 35 destinations

In [3]:
list_35_destinations = []

In [4]:
browser.get("https://www.booking.com/searchresults.fr.html?&ss=eyragues")
browser.maximize_window()
time.sleep(3)

In [ ]:


time.sleep(3)
rgpd_button = browser.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
rgpd_button.click()
time.sleep(2)

# text_box = browser.find_element(By.NAME,'ss')
# submit_button = browser.find_element(By.XPATH,'//*[@id="frm"]/div[1]/div[4]/div[2]/button')

# text_box.send_keys('avignon')
# submit_button.click()

sort_result_button = browser.find_element(By.XPATH, '//*[@id="search_results_table"]/div[2]/div/div/div/div[2]/span/button')
sort_result_button.click()                          
time.sleep(3)

etoiles_ordre_decroissant_button = browser.find_element(By.XPATH, '//*[@id="__bui-c3-1"]/div/div/div/div/ul/li[3]/button')
etoiles_ordre_decroissant_button.click()                            
time.sleep(5)


list_results = []
count=1


property_cards = browser.find_elements(By.CSS_SELECTOR, 'div[data-testid="property-card"]')

for property in property_cards:
    new_result = {}

    print(count)

    try:
        name = property.find_element(By.CSS_SELECTOR,'div[data-testid="title"]').text
    except:
        name = 'N/A'
    print(name)

    try:
        link = property.find_element(By.CSS_SELECTOR,'a[data-testid="title-link"]').get_attribute('href')
    except:
        link = 'N/A'
    print(link)

    try:
        review_score, _, review_count = property.find_element(By.CSS_SELECTOR, '[data-testid="review-score"]').text.split('\n')
    except:
        review_score = 'N/A' 
        review_count = 'N/A'
    print(review_score, review_count)

    try:
        area = property.find_element(By.CSS_SELECTOR, '[data-testid="address"]').text
    except:
        area = 'N/A'
    print(area)
    

    ##### RECORDING RESULTS AND OPEN A NEW WINDOW TO SCRAP MORE INFORMATIONS ########    

    new_result['name'] = name
    new_result['link'] = link
    new_result['review_score'] = review_score
    new_result['review_count'] = review_count
    #new_result['price'] = price
    new_result['area'] = area


    ##################

    # init browser2
    time.sleep(2)
    browser2 = webdriver.Chrome(chrome_options=options, service=path)
    browser2.get(link)
    browser2.maximize_window()
    time.sleep(5)

    #RGPD Button
    rgpd_button = browser2.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
    rgpd_button.click()
    time.sleep(3)

    try:
        address = browser2.find_element(By.XPATH, '//*[@id="showMap2"]/span[1]').text
    except:
        address = 'N/A'

    new_result['address'] = address
    print(address)
    time.sleep(1)

    try:
        coordinates_gps = browser2.find_element(By.XPATH, '//*[@id="showMap2"]/span[1]').get_attribute('data-bbox')
    except:
        coordinates_gps = 'N/A'
        
    new_result['coordinates_gps'] = coordinates_gps 
    print(coordinates_gps)
    print()
    time.sleep(1)

    try:
        button_see_more = browser2.find_element(By.XPATH, '//*[@id="basiclayout"]/div[1]/div[2]/div/div/div[1]/div[1]/div[1]/div[2]/a').click()
        time.sleep(2)
    except:
        button_see_more = 'N/A'
    
    try:
        description = browser2.find_element(By.XPATH, '//*[@id="basiclayout"]/div[1]/div[2]/div/div/div[1]/div[1]/div[1]').text
    except:
        description = 'N/A'

    new_result['description'] = description
    time.sleep(1)

    # close browser 2
    browser2.close()

    ##################


    # add all news informations in a list
    list_results.append(new_result)
   
    count +=1
    time.sleep(3)



time.sleep(10)
browser.close()



In [ ]:
list_results

In [7]:
df = pd.DataFrame(list_results)

In [8]:
df.head()

,name,link,review_score,review_count,area,address,coordinates_gps,description
0,"Maison tendance avec jardin, centre village, A...",https://www.booking.com/hotel/fr/les-mazets-de...,"8,7",12 expériences vécues,Eyragues,"9 Avenue de la République, 13630 Eyragues, France","4.81858249248091,43.8228958834269,4.8684907075...",Vous pouvez bénéficier d'une réduction Genius ...
1,Beautiful villa with air conditioning large pr...,https://www.booking.com/hotel/fr/bastide-prove...,N/A,N/A,Eyragues,"13630 Eyragues, France","4.82368835168188,43.8146235834269,4.8735896483...","Située à Eyragues, la Beautiful villa with air..."
2,La case d'Eyragues au coeur des Alpilles,https://www.booking.com/hotel/fr/la-case-d-eyr...,N/A,N/A,Eyragues,"1290 Chemin des Prés, 13630 Eyragues, France","4.82484003031537,43.8333224834269,4.8747569696...","Situé à Eyragues, à moins de 13 km de la gare ..."
3,Villa de prestige 10 personnes à Eyragues,https://www.booking.com/hotel/fr/villa-de-pres...,N/A,N/A,Eyragues,"chemin de la Gardy 231, 13630 Eyragues, France","4.81756506047357,43.8165159834269,4.8674679395...","Située à Eyragues, la Villa de prestige 10 per..."
4,Maison d'hôtes Le Pré Gourmand - Adults only,https://www.booking.com/hotel/fr/le-pre-gourma...,"9,1",82 expériences vécues,Eyragues,"175 avenue marx dormoy, 13630 Eyragues, France","4.81740848965353,43.8262495834269,4.8673195103...","Située à Eyragues, à 13 km de la gare centrale..."


In [9]:
timestr = time.strftime("%Y%m%d-%H%M%S")
df.to_csv(f'save_output/{timestr}.csv', index=False)



In [10]:
# Serializing json
json_object = json.dumps(list_results)

# Writing to sample.json
with open(f"save_output/{timestr}.json", "w") as outfile:
    outfile.write(json_object)